In [ ]:
import os
import json

import sys
sys.path.append('./tools/client')
import prometheus
import grafana
import config_parser

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def collect_data(start_time, end_time, collect_cfg):
    grafana_auth = collect_cfg["grafana"]["auth"]
    grafana_server = collect_cfg["grafana"]["server"]
    prom_server = collect_cfg["prometheus"]["server"]
    aio_db = collect_cfg["query"]["dashboard"]
    step = collect_cfg["query"]["step"]
    dash_boards = collect_cfg["query"]["dashboard"]
    
    pclient = prometheus.client(prom_server)
    gclient = grafana.client(grafana_server, grafana_auth)
    
    db_datas = [gclient.get_db(db) for db in dash_boards]
    assert len(db_datas) > 0, "no prometheus data collect"
    
    if "aio" in dash_boards:
        logging.warning("while aio dashboard is existing, only fetching aio data")
        dash_boards = ["aio"]
    
    targets = []
    if len(dash_boards) == 1 and dash_boards[0] == "aio":
        # assert it is aio
        targets = db_datas[0]["panels"][0]["targets"]
    else:
        for db_data in db_datas:
            targets = targets + config_parser.read_targets_from_json(db_data)
    
    df = pclient.targets_to_df(targets, start_time, end_time, step)
    print(df.info())
    return df

def refetch_data(exp_dir):
    
    collect_cfg = {
        "grafana":{
            "auth": "admin:admin@",
            "server": "10.208.130.243:3001",},
        "prometheus":{
            "server": "10.208.130.243:9090",},
        "query":{
            "dashboard":["aio"],
            "step": "4s",},
        "save_file": "merged.csv", 
    }
    
    
    exp_json = os.path.join(exp_dir, "exp.json")
    with open(exp_json, 'r') as f:
        exp_data = json.load(f)
        
    df = collect_data(exp_data["start_time"], exp_data["end_time"], collect_cfg)
    df.to_csv(os.path.join(exp_dir, "merged.csv"))    

In [ ]:
root_dir = "/home/ict/appProfile/data/same_numa"
exp_dirs  = [os.path.join(root_dir, dir) for dir in os.listdir(root_dir)]
for exp_dir in exp_dirs:
    refetch_data(exp_dir)